In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install beautifulsoup4

In [3]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 6.9 MB/s eta 0:00:00


In [7]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3


In [2]:
!pip install torch==2.6.0

In [5]:
import os
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel
from torch.optim import AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [6]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# Download the missing 'punkt_tab' data package
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
df_train = pd.read_csv("/content/drive/MyDrive/Projects/Metahuman Chatbot/sentian_cnn/GoEmotions/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv("/content/drive/MyDrive/Projects/Metahuman Chatbot/sentian_cnn/GoEmotions/test.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [8]:
df_train['List of classes'] = df_train['Class'].apply(lambda x: x.split(','))
df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))
df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))

In [9]:
with open('/content/drive/MyDrive/Projects/Metahuman Chatbot/sentian_cnn/GoEmotions/6/data/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

In [10]:
emotion_file = open("/content/drive/MyDrive/Projects/Metahuman Chatbot/sentian_cnn/GoEmotions/6/data/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [11]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [12]:
df_train['Emotions'] = df_train['List of classes'].apply(idx2class)
df_dev['Emotions'] = df_dev['List of classes'].apply(idx2class)

In [13]:
def EmotionMapping(emotion_list):
    map_list = []

    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('anger')
        if i in ekman_mapping['disgust']:
            map_list.append('disgust')
        if i in ekman_mapping['fear']:
            map_list.append('fear')
        if i in ekman_mapping['joy']:
            map_list.append('joy')
        if i in ekman_mapping['sadness']:
            map_list.append('sadness')
        if i in ekman_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')

    return map_list

In [14]:
df_train['Mapped Emotions'] = df_train['Emotions'].apply(EmotionMapping)
df_dev['Mapped Emotions'] = df_dev['Emotions'].apply(EmotionMapping)

In [15]:
df_train['anger'] = np.zeros((len(df_train),1))
df_train['disgust'] = np.zeros((len(df_train),1))
df_train['fear'] = np.zeros((len(df_train),1))
df_train['joy'] = np.zeros((len(df_train),1))
df_train['sadness'] = np.zeros((len(df_train),1))
df_train['surprise'] = np.zeros((len(df_train),1))
df_train['neutral'] = np.zeros((len(df_train),1))

df_dev['anger'] = np.zeros((len(df_dev),1))
df_dev['disgust'] = np.zeros((len(df_dev),1))
df_dev['fear'] = np.zeros((len(df_dev),1))
df_dev['joy'] = np.zeros((len(df_dev),1))
df_dev['sadness'] = np.zeros((len(df_dev),1))
df_dev['surprise'] = np.zeros((len(df_dev),1))
df_dev['neutral'] = np.zeros((len(df_dev),1))

In [16]:
for i in ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise','neutral']:
    df_train[i] = df_train['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    df_dev[i] = df_dev['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

In [17]:
df_train.head()

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,0,0,0,0,1
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[anger],1,0,0,0,0,0,0
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[fear],0,0,1,0,0,0,0
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[anger],1,0,0,0,0,0,0


In [18]:
df_dev.head()

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,anger,disgust,fear,joy,sadness,surprise,neutral
0,I’m really sorry about your situation :( Altho...,25,eecwqtt,[25],1,[sadness],[sadness],0,0,0,0,1,0,0
1,It's wonderful because it's awful. At not with.,0,ed5f85d,[0],1,[admiration],[joy],0,0,0,1,0,0,0
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3,[13],1,[excitement],[joy],0,0,0,1,0,0,0
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1,[15],1,[gratitude],[joy],0,0,0,1,0,0,0
4,They got bored from haunting earth for thousan...,27,eem5uti,[27],1,[neutral],[neutral],0,0,0,0,0,0,1


In [19]:
df_train.drop(df_train[df_train['neutral'] == 1].index, inplace=True)
df_dev.drop(df_dev[df_dev['neutral'] == 1].index, inplace=True)
df_train.drop(df_train[df_train['disgust'] == 1].index, inplace=True)
df_dev.drop(df_dev[df_dev['disgust'] == 1].index, inplace=True)

In [20]:
df_train.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions', 'neutral', 'disgust'], axis=1, inplace=True)
df_dev.drop(['Class', 'List of classes', 'Len of classes', 'Emotions', 'Mapped Emotions', 'neutral', 'disgust'], axis=1, inplace=True)

In [21]:
def nltk_preprocessing_pipeline(text):
    # Convert emojis to text
    text = emoji.demojize(text)
    # Lowercase the text
    text = text.lower()
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove HTML tags
    text = BeautifulSoup(text, 'lxml').get_text()
    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    # Tokenize using NLTK
    tokens = word_tokenize(text)
    # Remove punctuation and non-alphanumeric tokens
    tokens = [word for word in tokens if word.isalnum()]
    # Lemmatize tokens using WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Reconstruct the text from tokens
    processed_text = " ".join(tokens)
    return processed_text

In [22]:
# Apply the updated NLTK preprocessing pipeline to both training and development data
df_train['Text'] = df_train['Text'].apply(nltk_preprocessing_pipeline)
df_dev['Text'] = df_dev['Text'].apply(nltk_preprocessing_pipeline)

In [23]:
df_train.reset_index(drop=True).to_csv("train.csv", index=False)
df_dev.reset_index(drop=True).to_csv("val.csv", index=False)

In [24]:
df_train = df_train.reset_index(drop=True)
df_dev = df_dev.reset_index(drop=True)

In [25]:
df_train.head()

,Text,ID,anger,fear,joy,sadness,surprise
0,why the fuck is bayless isoing,eezlygj,1,0,0,0,0
1,to make her feel threatened,ed7ypvh,0,1,0,0,0
2,dirty southern wanker,ed0bdzj,1,0,0,0,0
3,omg peyton is good enough to help u in the pla...,edvnz26,0,0,0,0,1
4,yes i heard abt the f bomb that ha to be why t...,ee3b6wu,0,0,1,0,0


In [26]:
print(df_train.shape)
print(df_dev.shape)

(28427, 7)
(3520, 7)


In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [29]:
target_cols = [col for col in df_train.columns if col not in ['Text', 'ID']]
target_cols

['anger', 'fear', 'joy', 'sadness', 'surprise']

In [30]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [31]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(df_dev, tokenizer, MAX_LEN)

In [32]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE,
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE,
                          num_workers=4, shuffle=False, pin_memory=True)

In [33]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,5)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

model = BERTClass()
model.to(device);

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [35]:
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  0.6878846287727356
Epoch: 1, Loss:  0.161261186003685
Epoch: 2, Loss:  0.1334858387708664
Epoch: 3, Loss:  0.10946585983037949
Epoch: 4, Loss:  0.12297411262989044
Epoch: 5, Loss:  0.11308642476797104
Epoch: 6, Loss:  0.050859399139881134
Epoch: 7, Loss:  0.07135267555713654
Epoch: 8, Loss:  0.07797227799892426
Epoch: 9, Loss:  0.04310990124940872


In [ ]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.743465909090909
F1 Score (Micro) = 0.8110964332892999
F1 Score (Macro) = 0.7439324873867977


In [ ]:
# save_path = '/content/drive/MyDrive/Projects/Metahuman Chatbot/sentian_cnn/GoEmotions/model.bin'
# torch.save(model.state_dict(), save_path)


In [ ]:
torch.save(model.state_dict(), 'model.bin')

In [36]:
from transformers import AutoConfig, AutoModel

In [37]:
config = AutoConfig.from_pretrained('bert-base-uncased')
model =  AutoModel.from_config(config)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [39]:
from transformers import AutoConfig, AutoModel
import torch

# Load the model architecture that was used to save the weights
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,5)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

# Instantiate the model with the correct architecture
model = BERTClass()


PATH = "/content/drive/MyDrive/Projects/Metahuman Chatbot/sentian_cnn/Outputs/model_nltk.bin"
# Load the state dict into the model
model.load_state_dict(torch.load(PATH))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [40]:
from transformers import AutoTokenizer
import torch

# Load the tokenizer for 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Example text to classify
text = "Your sample text goes here."

# Tokenize the input text
encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,            # You can adjust this based on your model's requirements
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

# Extract the inputs
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# For Roberta, token_type_ids may not be provided, so we create a dummy tensor if needed.
token_type_ids = encoding.get('token_type_ids')
if token_type_ids is None:
    token_type_ids = torch.zeros_like(input_ids)

# Set the model to evaluation mode
model.eval()

# Perform the prediction without calculating gradients
with torch.no_grad():
    outputs = model(input_ids, attention_mask, token_type_ids)

# Get the predicted class by taking the index of the maximum logit
predicted_class = torch.argmax(outputs, dim=1).item()

print("Predicted class:", predicted_class)


Predicted class: 2


In [41]:
from transformers import AutoTokenizer
import torch

# Load the tokenizer for 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Example text to classify
text = "Shit bro the condom broke"

# Define a mapping of class indices to class names
class_names = {
    0: "Anger",
    1: "Fear",
    2: "Joy",
    3: "Sadness",
    4: "Surprise"
}

# Tokenize the input text
encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,     # Adjust based on your needs
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
token_type_ids = encoding.get('token_type_ids')
if token_type_ids is None:
    token_type_ids = torch.zeros_like(input_ids)

# Set model to evaluation mode
model.eval()

# Perform prediction without gradient tracking
with torch.no_grad():
    outputs = model(input_ids, attention_mask, token_type_ids)

# Get the predicted index
predicted_index = torch.argmax(outputs, dim=1).item()

# Map the predicted index to the class name
predicted_label = class_names[predicted_index]

print("Predicted label:", predicted_label)


Predicted label: Sadness
